In [25]:
! pip install -q langchain-community langchain-core langchain-google-genai langsmith

In [26]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os

In [27]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
LANGCHAIN_API_KEY = userdata.get('LANGCHAIN_API_KEY')
LANGCHAIN_PROJECT = userdata.get('LANGCHAIN_PROJECT')
LANGCHAIN_TRACING_V2 = "true"

In [28]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT
os.environ["LANGCHAIN_TRACING_V2"] = LANGCHAIN_TRACING_V2

In [29]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [30]:
print(model.invoke("Hi").content)

Hi there! How can I help you today?


In [31]:
parser = StrOutputParser()
parser.invoke(model.invoke("Hi"))

'Hi there! How can I help you today?'

In [32]:
while True:
  user_input = input("Enter input:")
  if user_input != "exit":
    print(model.invoke(user_input))
  else:
    break


Enter input:hi
content='Hi there! How can I help you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-a8a0def5-407b-41ca-aee8-d793e6ef22eb-0' usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}}
Enter input:exit


In [33]:
output = model.invoke(
    [
        ("user","Hi my name is saurav"),
        ("ai","Hello Saurav, How can i assist you?"),
        ("user","What is my name")
    ]
)

In [34]:
parser.invoke(output)

'Your name is Saurav.'

In [35]:
store = {}

def get_session_id(session_id) -> BaseChatMessageHistory:
  if session_id not in store:
    store[session_id] = InMemoryChatMessageHistory()
  return store[session_id]

In [36]:
model_with_memory = RunnableWithMessageHistory(model,get_session_id)

In [37]:
model_with_memory.invoke("Hi, my name is saurav",config={"session_id":"session1"})

AIMessage(content="Hi Saurav, it's nice to meet you!  How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-0066ebfd-e41a-41f1-a420-0225f56e2717-0', usage_metadata={'input_tokens': 8, 'output_tokens': 21, 'total_tokens': 29, 'input_token_details': {'cache_read': 0}})

In [38]:
model_with_memory.invoke("What is my name",config={"session_id":"session1"})

AIMessage(content='Your name is Saurav.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-121f214f-a88d-414a-8c05-423b5d8b4dba-0', usage_metadata={'input_tokens': 34, 'output_tokens': 7, 'total_tokens': 41, 'input_token_details': {'cache_read': 0}})

In [39]:
model_with_memory.invoke("Hi, my name is raj",config={"session_id":"session2"})

AIMessage(content="Hello Raj, it's nice to meet you!  How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-b0c747ef-c076-40fd-9133-4941f9720e15-0', usage_metadata={'input_tokens': 7, 'output_tokens': 20, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}})

In [40]:
model_with_memory.invoke("what is my name",config={"session_id":"session2"})

AIMessage(content='Your name is Raj.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-173f6fd0-8c36-4127-8439-182075a1ccbe-0', usage_metadata={'input_tokens': 32, 'output_tokens': 6, 'total_tokens': 38, 'input_token_details': {'cache_read': 0}})

In [41]:
store

{'session1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, my name is saurav', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Saurav, it's nice to meet you!  How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-0066ebfd-e41a-41f1-a420-0225f56e2717-0', usage_metadata={'input_tokens': 8, 'output_tokens': 21, 'total_tokens': 29, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Saurav.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-121f214f-a88d-414a-8c05-423b5d8b4dba-0', usage_metadata={'input_tokens': 34, 'output_tokens': 7, 'total_tokens': 41, 'input_token_details': {'cache_read': 0}})]),
 'sessio

In [42]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a nice AI bot that helps a user figure out what to eat in one short sentence"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [43]:
chain = prompt | model | parser

In [44]:
chain.invoke({"messages":["hi my name is saurav"]})

'Hi Saurav, what kind of food are you in the mood for?'

In [45]:
chain.invoke({"messages":["what is my name"]})

'I do not know your name, but I can help you decide what to eat!'

In [46]:
model_with_memory = RunnableWithMessageHistory(chain,get_session_id)

In [47]:
model_with_memory.invoke("Hi my name is saurav",config={"session_id":"session1"})

'Hi Saurav!  What would you like to eat?'

In [48]:
model_with_memory.invoke("what is my name?",config={"session_id":"session1"})

'Your name is Saurav.'